In [1]:
import tweepy #https://github.com/tweepy/tweepy
import csv

DEBUG = False

#Twitter API credentials
consumer_key = ""
consumer_secret = ""
access_key = ""
access_secret = ""


def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    user=api.get_user(screen_name=screen_name)
    
    #retweets=([t.user.screen_name for t in new_tweets[0].retweets()])
    
    #return
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        if DEBUG: print "getting tweets before %s" % (oldest)
        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,
                                       count=200,max_id=oldest)
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        if DEBUG: print "...%s tweets downloaded so far" % (len(alltweets))
    
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [{'id_str' : tweet.id_str, 
                  'created_at' : tweet.created_at, 
                  'text' : tweet.text.encode("utf-8"),
                  'retweet_count' : tweet.retweet_count,
                  'screen_name' : screen_name}
                 
                 for tweet in alltweets]
    
    
    return outtweets
    
    #write the csv    
#     with open('%s_tweets.csv' % screen_name, 'wb') as f:
#         writer = csv.writer(f)
#         writer.writerow(["id","created_at","text", "retweet_count", "retweet_user_names"])
#         writer.writerows(outtweets)
    
#     pass

tweets = get_all_tweets("jkcorden")

In [2]:
len(tweets)

3240

In [3]:
original_tweets = [t for t in tweets if t['text'][:2] != 'RT']

In [4]:
len(original_tweets)

1348

In [5]:
ids = [tw["id_str"] for tw in original_tweets]

In [6]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [7]:
new_ids=[]
for i in ids:
    new_ids.append(i.encode('utf-8'))
new_ids


['836950315416989697',
 '836821102173237248',
 '836799639986843648',
 '836768307793387520',
 '836671532818542592',
 '836599408997224448',
 '836469704445186048',
 '836467564913934336',
 '836311189369991168',
 '835679473341251584',
 '835655946789597184',
 '835218983171153921',
 '835168308475224064',
 '835140241170128896',
 '834885554625863680',
 '834852107534557184',
 '834841866776375296',
 '834500556051804160',
 '834115053343289344',
 '832797864824705025',
 '831314150831697920',
 '831261843003826176',
 '831229522770997248',
 '831029977323753473',
 '830846729075380224',
 '830566328331182080',
 '830436778116984833',
 '830253267175165952',
 '830243251382689792',
 '830234244936921089',
 '830209742228254720',
 '829896642211901440',
 '829785357621157888',
 '829785258295832577',
 '829760606559313921',
 '829752227971559425',
 '829447650613211136',
 '829388228696305664',
 '829386453486407680',
 '829377352752783361',
 '829371804141490176',
 '829370917482475524',
 '829155908852723712',
 '829123220

In [8]:
import time

In [9]:
len(new_ids)

1348

In [27]:
def id_sn_location(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
            
        with open('jamesCorden300.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [28]:
#id_sn_location(new_ids[0:300])

In [29]:
def id_sn_location1000(ids):
    RTS=[]
    for i in ids:
        try:
            RTS.append((i, api.get_status(i).retweets()))
        
            # Some tweepy api call, ex) api.get_user(screen_name = usrScreenName)
        except tweepy.TweepError as e:
            pass
        time.sleep(13)
            
        with open('jamesCorden1000.csv','wb') as f:
            writer = csv.writer(f)
            retweet=[]
            for i, t in RTS:
                for rts in t:
                    retweet.append([i,
                                    rts.user.screen_name.encode('utf-8'),
                                    rts.user.location.encode('utf-8')])
            writer.writerow(['id','screen_name', 'location'])
            writer.writerows(retweet)

In [30]:
#id_sn_location1000(new_ids[1001:])

In [31]:
import pandas as pd

In [32]:
jamescorden=pd.read_csv('jamesCorden300.csv')
jamescorden301=pd.read_csv('jamesCorden2.csv')
jamescorden1000=pd.read_csv('jamesCorden1000.csv')

In [33]:
jamescorden.head()

,id,screen_name,location
0,836950315416989697,msopinionated76,NaN
1,836950315416989697,004clinden,"Austin, TX"
2,836950315416989697,Dmusee,"LA, CA"
3,836950315416989697,nicksfan2,st louis mo\n
4,836950315416989697,kute_asabutton,in a landfill of fandoms


In [34]:
jamescorden301.head()

,id,screen_name,location
0,776170839473467392,OlivialodrLambe,NaN
1,776170839473467392,Akilleazee,NaN
2,776170839473467392,jmcinnis84,NaN
3,776170839473467392,jakequenichet,NaN
4,776170839473467392,scolefurlong,NaN


In [35]:
jamescorden1000.head()

,id,screen_name,location
0,668633731872133120,bflbfth,lon of the don
1,668633731872133120,A_ZHNLL,México
2,668633731872133120,dumplin50,NaN
3,668633731872133120,emilyneedachill,NaN
4,668633731872133120,Mcghee011,Aberdeen Scotland


In [81]:
jamescorden2=[jamescorden, jamescorden301, jamescorden1000]

In [82]:
thejamescorden=pd.concat(jamescorden2)
thejamescorden

,id,screen_name,location
0,836950315416989697,msopinionated76,NaN
1,836950315416989697,004clinden,"Austin, TX"
2,836950315416989697,Dmusee,"LA, CA"
3,836950315416989697,nicksfan2,st louis mo\n
4,836950315416989697,kute_asabutton,in a landfill of fandoms
5,836950315416989697,MichaelMick777,USA - #LGBT -
6,836950315416989697,melanieromero23,"Los Angeles, CA"
7,836950315416989697,ztk92,"Sterling, VA"
8,836950315416989697,___FRB,Puerto Rico
9,836950315416989697,louisambayon,ARG【tresdemayo/2014】


In [84]:
thejamescorden.to_pickle('jamescorden.pkl')

In [83]:
len(thejamescorden['id'].unique())

1344

In [39]:
test=thejamescorden.groupby(['id'], as_index=False).agg(lambda x: set(x))

In [40]:
test2=test.drop(['location'], axis=1)

In [41]:
test3=test2['screen_name'].str.join(sep='*').str.get_dummies(sep='*')

In [42]:
test4=pd.concat([test2, test3], axis=1)

In [43]:
test4.head()

,id,screen_name,004clinden,007ayremick,007posie,008DAnia,00StarEngel00,00_hurtado,0161Darren,02Sofiyka02,...,zsm_crystal,ztj521,ztk92,zulazry,zuri_horan1502,zxyngublxr,zyjmcrew,zzafariz,zzaynlessly,zzfYiuEe4a4Sjui
0,629381068060164096,"{FinleyPlummer_, BigmikeEDL71, Ollie_higgs, ch...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,629500127560925184,"{takethatno1fanx, gillianbarlowx, thecircus__,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,629525736299212800,"{emilynahanee, TeamJKCORDEN, starshipjergi, li...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,629666065920888832,"{TheJKCrew, FallonJokersSM, SueMasterson1, gee...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,629666249513922562,"{charlierange, TeamJKCORDEN, suerowleytweets, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
test4.sum(axis=0)

id                 7399627121149837915
004clinden                           8
007ayremick                          1
007posie                             1
008DAnia                             1
00StarEngel00                        1
00_hurtado                           1
0161Darren                           1
02Sofiyka02                          3
02lopezCarolina                      1
0405zahraa                           1
0504st                               3
05SingingIdiots                      1
07_alex13                            1
07a4077b21b842a                      2
0822230710                           3
09ShawN03                            1
0IKlt8KqCInJ5FH                      1
0Laeti                               1
0MG1D                                2
0dd7qjkvxK4lvvc                      1
0fMiceEtAkira                        1
0g5jywcoiBBe4nO                      1
0healthyeating0                      1
0ttaM                                2
100degreebot             

In [45]:
test4.sum(axis=0).sort_values(ascending=False)

id                 7399627121149837915
TeamJKCORDEN                       510
awlasky                            456
charlierange                       327
MrCoolLooking75                    197
TheJKCrew                          196
pardeepinsa19                       94
ConnieOnUnicorn                     88
HeavyIsAlive                        88
JamesCordenGuat                     85
yazenalharethi                      84
CordenNEWS                          84
mrsbmag                             81
SoCalPalKim                         76
A16official                         71
Lovatic4_LouisT                     68
JacquiRoberts13                     60
phucviet011                         55
MMiyu1                              55
VictoriaYaez3                       51
GobeyyP                             50
Onionlover01                        49
Tash_Beliber_xx                     48
BeckyTesoro                         45
MarOfDirection                      45
DannyJoline              

In [46]:
tweets2 = get_all_tweets("TeamJKCORDEN ")
tweets3 = get_all_tweets("awlasky")
tweets4 = get_all_tweets("charlierange")

In [47]:
tweets5 = get_all_tweets("MrCoolLooking75")
tweets6 = get_all_tweets("TheJKCrew")
tweets7 = get_all_tweets("pardeepinsa19")

In [48]:
tweets8 = get_all_tweets("ConnieOnUnicorn")
tweets9 = get_all_tweets("HeavyIsAlive")
tweets10 = get_all_tweets("JamesCordenGuat")
tweets11 = get_all_tweets("yazenalharethi")

In [49]:
print len(tweets2)
print len(tweets3)
print len(tweets4)
print len(tweets5)
print len(tweets6)
print len(tweets7)
print len(tweets8) 
print len(tweets9) 
print len(tweets10)
print len(tweets11)

3188
3249
3223
3207
3218
3202
3223
3222
263
3236


In [50]:
ogt1 = [t for t in tweets2 if t['text'][:2] != 'RT'] 
ogt2 = [t for t in tweets3 if t['text'][:2] != 'RT']
ogt3 = [t for t in tweets4 if t['text'][:2] != 'RT']
ogt4 = [t for t in tweets5 if t['text'][:2] != 'RT']
ogt5 = [t for t in tweets6 if t['text'][:2] != 'RT']
ogt6 = [t for t in tweets7 if t['text'][:2] != 'RT']
ogt7 = [t for t in tweets8 if t['text'][:2] != 'RT']
ogt8 = [t for t in tweets9 if t['text'][:2] != 'RT']
ogt9 = [t for t in tweets10 if t['text'][:2] != 'RT']
ogt10 = [t for t in tweets11 if t['text'][:2] != 'RT']

In [52]:
print len(tweets2), len(ogt1) 
print len(tweets3), len(ogt2) 
print len(tweets4), len(ogt3) 
print len(tweets5), len(ogt4) 
print len(tweets6), len(ogt5) 
print len(tweets7), len(ogt6) 
print len(tweets8) , len(ogt7) 
print len(tweets9) , len(ogt8) 
print len(tweets10), len(ogt9) 
print len(tweets11), len(ogt10)

3188 380
3249 1008
3223 551
3207 12
3218 1472
3202 13
3223 533
3222 428
263 5
3236 362


In [54]:
jamesC_tweets=pd.DataFrame(original_tweets)
jamesC_tweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-01 14:44:39,836950315416989697,151,jkcorden,Last night @KendallJenner Andrew Garfield @Ste...
1,2017-03-01 06:11:12,836821102173237248,6,jkcorden,@dukeanyanwu Thanks man x
2,2017-03-01 04:45:55,836799639986843648,130,jkcorden,"CBS Is launching a new show tonight, 'Hidden T..."
3,2017-03-01 02:41:25,836768307793387520,108,jkcorden,Here's @realDonaldTrump practicing a Carpool K...
4,2017-02-28 20:16:52,836671532818542592,3391,jkcorden,I was originally meant to be with @Louis_Tomli...


In [55]:
import re

In [56]:
jamesC_words=[]
for i in jamesC_tweets['text']:
    jamesC_words.append(re.sub(r"http\S+","", i))
jamesC_words

['Last night @KendallJenner Andrew Garfield @StevieNicks  and Taylor Lautner showed their Hidden Talents!',
 '@dukeanyanwu Thanks man x',
 "CBS Is launching a new show tonight, 'Hidden Talents' featuring @KendallJenner Andrew Garfield, @StevieNicks and Taylor Lautner @cbs 12.35 \xf0\x9f\x98\x98",
 "Here's @realDonaldTrump practicing a Carpool Karaoke on his way tonight",
 'I was originally meant to be with @Louis_Tomlinson and @steveaoki on just hold on. Here is video evidence! ',
 'Just to be clear, this is me. NOT Emma Stone. Just to clear that up x ',
 "Here's to the ones who lose... ",
 'We opened our show tonight with the audition scene from @LaLaLand ',
 'Such fun!!!! ',
 "I genuinely can't stop watching @mulaney and @nickkroll crush this!!!! ",
 'This is a good a monologue at an awards show as you will ever see. Brilliant! ',
 'This is madness.  ',
 'This hardcore boyband is NSFW ',
 'The worlds first hardcore boyband ',
 '',
 "It's an outrage that Ranieri has been sacked! Crazy

In [57]:
retweeters1=pd.DataFrame(tweets2)
retweeters2=pd.DataFrame(tweets3)
retweeters3=pd.DataFrame(tweets4)
retweeters4=pd.DataFrame(tweets5)
retweeters5=pd.DataFrame(tweets6)
retweeters6=pd.DataFrame(tweets7)
retweeters7=pd.DataFrame(tweets8)
retweeters8=pd.DataFrame(tweets9)
retweeters9=pd.DataFrame(tweets10)
retweeters10=pd.DataFrame(tweets11)

In [58]:
frames=[retweeters1,retweeters2,
    retweeters3,retweeters4,
    retweeters5,retweeters6,
    retweeters7,retweeters8,
    retweeters9,retweeters10]

In [59]:
all_retweets=pd.concat(frames)
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-02 21:58:50,837421969196920832,1,TeamJKCORDEN,"@JKCorden is joined on @latelateshow tonight, ..."
1,2017-03-02 21:20:15,837412258183593984,1,TeamJKCORDEN,@JKCorden on @ChrisMoyles show at @BBCR1 studi...
2,2017-03-02 20:47:40,837404059850194945,0,TeamJKCORDEN,@JKCorden recreates #LALALAND jokes about Osca...
3,2017-03-02 20:46:49,837403846251008000,0,TeamJKCORDEN,Meet @JKCorden's sex-crazed boyband called Thr...
4,2017-03-02 20:46:07,837403667577851904,0,TeamJKCORDEN,"@JKCorden and wife, Julia, arrive at #TheOscar..."


In [60]:
clean_text=[]
for i in all_retweets['text']:
    clean_text.append(re.sub(r"http\S+","", i))

In [61]:
no_RT=[]
for i in clean_text:
    no_RT.append(re.sub(r"RT", "", i))

In [62]:
retweet_text=pd.DataFrame(no_RT, )

In [63]:
all_retweets['text']=retweet_text
all_retweets.head()

,created_at,id_str,retweet_count,screen_name,text
0,2017-03-02 21:58:50,837421969196920832,1,TeamJKCORDEN,"@JKCorden is joined on @latelateshow tonight, ..."
1,2017-03-02 21:20:15,837412258183593984,1,TeamJKCORDEN,@JKCorden on @ChrisMoyles show at @BBCR1 studi...
2,2017-03-02 20:47:40,837404059850194945,0,TeamJKCORDEN,@JKCorden recreates #LALALAND jokes about Osca...
3,2017-03-02 20:46:49,837403846251008000,0,TeamJKCORDEN,Meet @JKCorden's sex-crazed boyband called Thr...
4,2017-03-02 20:46:07,837403667577851904,0,TeamJKCORDEN,"@JKCorden and wife, Julia, arrive at #TheOscar..."


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.corpus import stopwords

In [75]:
stopset = set(stopwords.words('english'))
stopset.update(stopwords.words('spanish'))
stopset.update(['love', 'like', 'new', 'amp', 'get', 'one', 'see', 'si', 'today', 'tonight',
               'go','want','good', 'back', 'thank', 'us', 'day','people','make',
                'much', 'know',])

In [76]:
retweets_str=','.join(no_RT)

In [77]:
vectorizer=TfidfVectorizer(stop_words=stopset,use_idf=True, ngram_range=(1,3))
X=vectorizer.fit_transform([retweets_str])

In [78]:
lsa=TruncatedSVD(n_components=1, n_iter=100, random_state=42)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=1, n_iter=100,
       random_state=42, tol=0.0)

In [85]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:20]
    print("James Corden %d:" % i )
    for term in sortedTerms:
        print(term[0])
    print (" ")

James Corden 0:
jkcorden
latelateshow
james
gurmeetramrahim
awlasky
corden
james corden
show
watch
night
time
silvanapetrucci
thanks
thejkcrew
please
dannyfe76914527
best
great
fessiejant
cbearfalpal
 
